#### #translation-encoder
#### #translation-transformer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Model
from keras.layers import Input,LSTM, Dense


In [2]:
data_path = [
    ("Go.", "Va !"),
    ("Run!", "Cours !"),
    ("Run.", "Cours !"),
    ("Who?", "Qui ?"),
    ("Wow!", "Ça alors !"),
    ("Fire!", "Au feu !"),
    ("Help!", "À l'aide !"),
    ("Stop!", "Arrête-toi !"),
    ("Wait!", "Attends !"),
    ("Hello!", "Bonjour !"),
    ("I see.", "Je comprends.")

]

### Data processing

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for input_text, target_text in data_path:
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
target_texts

['\tVa !\n',
 '\tCours !\n',
 '\tCours !\n',
 '\tQui ?\n',
 '\tÇa alors !\n',
 '\tAu feu !\n',
 "\tÀ l'aide !\n",
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tBonjour !\n',
 '\tJe comprends.\n']

In [5]:
input_characters

{' ',
 '!',
 '.',
 '?',
 'F',
 'G',
 'H',
 'I',
 'R',
 'S',
 'W',
 'a',
 'e',
 'h',
 'i',
 'l',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'w'}

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

num_encoder_tokens, num_decoder_tokens

(24, 33)

In [7]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
max_encoder_seq_length, max_decoder_seq_length

(6, 15)

In [8]:
# create token mapping (char -> int)

input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])


input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '.': 2,
  '?': 3,
  'F': 4,
  'G': 5,
  'H': 6,
  'I': 7,
  'R': 8,
  'S': 9,
  'W': 10,
  'a': 11,
  'e': 12,
  'h': 13,
  'i': 14,
  'l': 15,
  'n': 16,
  'o': 17,
  'p': 18,
  'r': 19,
  's': 20,
  't': 21,
  'u': 22,
  'w': 23},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  "'": 4,
  '-': 5,
  '.': 6,
  '?': 7,
  'A': 8,
  'B': 9,
  'C': 10,
  'J': 11,
  'Q': 12,
  'V': 13,
  'a': 14,
  'c': 15,
  'd': 16,
  'e': 17,
  'f': 18,
  'i': 19,
  'j': 20,
  'l': 21,
  'm': 22,
  'n': 23,
  'o': 24,
  'p': 25,
  'r': 26,
  's': 27,
  't': 28,
  'u': 29,
  'À': 30,
  'Ç': 31,
  'ê': 32})

In [9]:
reverse_input_token_index = dict([(i,char) for i, char in enumerate(input_characters)])
reverse_target_token_index = dict([(i,char) for i,char in enumerate(target_characters)])

reverse_input_token_index, reverse_target_token_index

({0: ' ',
  1: '!',
  2: '.',
  3: '?',
  4: 'F',
  5: 'G',
  6: 'H',
  7: 'I',
  8: 'R',
  9: 'S',
  10: 'W',
  11: 'a',
  12: 'e',
  13: 'h',
  14: 'i',
  15: 'l',
  16: 'n',
  17: 'o',
  18: 'p',
  19: 'r',
  20: 's',
  21: 't',
  22: 'u',
  23: 'w'},
 {0: '\t',
  1: '\n',
  2: ' ',
  3: '!',
  4: "'",
  5: '-',
  6: '.',
  7: '?',
  8: 'A',
  9: 'B',
  10: 'C',
  11: 'J',
  12: 'Q',
  13: 'V',
  14: 'a',
  15: 'c',
  16: 'd',
  17: 'e',
  18: 'f',
  19: 'i',
  20: 'j',
  21: 'l',
  22: 'm',
  23: 'n',
  24: 'o',
  25: 'p',
  26: 'r',
  27: 's',
  28: 't',
  29: 'u',
  30: 'À',
  31: 'Ç',
  32: 'ê'})

In [10]:
# Generate one hot encoded data
# Encoder Input hai -> (num of samples, max len, unique chars) #output dekh na lvde

encoder_data_input = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens))
encoder_data_input.shape

(11, 6, 24)

In [11]:
# Decoders Input -> (num of samples, max len, unique chars) #output dekh na lvde
decoder_data_input = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))
decoder_data_input.shape

(11, 15, 33)

In [12]:
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens))
decoder_target_data.shape

(11, 15, 33)

In [15]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_data_input[i, t, input_token_index[char]] = 1
    encoder_data_input[i, t+1:, input_token_index[' ']] = 1  # for padding

    for r, char in enumerate(target_text):
        decoder_data_input[i,t,target_token_index[char]] = 1
        if t>0:
            decoder_data_input[i, t-1:, target_token_index[char]] = 1
    decoder_data_input[i, t+1:, target_token_index[' ']] = 1  # for padding
    decoder_target_data[i, t:, target_token_index[' ']] = 1  # for padding

In [16]:
decoder_data_input

array([[[0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0.

In [20]:
# create encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder(encoder_inputs)

(<KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_9>,
 <KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_10>,
 <KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_11>)

In [21]:
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_outputs, state_h, state_c

(<KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_12>,
 <KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_13>,
 <KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_14>)

In [27]:
# create decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_state=True, return_sequences=True)
decoder_lstm(decoder_inputs)

(<KerasTensor shape=(None, None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_30>,
 <KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_31>,
 <KerasTensor shape=(None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_32>)

In [30]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = [state_h, state_c])
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [33]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary();

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None, 24)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None, 33)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    287,744 │ input_layer_2[0]… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, None,     │    296,960 │ input_layer_5[0]… │
│                     │ 256), (None,      │            │ lstm_2[3][1],     │
│                     │ 256), (None,      │            │ lstm_2[3][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 33)  │      8,481 │ lstm_5[3][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 593,185 (2.26 MB)

 Trainable params: 593,185 (2.26 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']);
model.fit([encoder_data_input, decoder_data_input], decoder_target_data, epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7576 - loss: 0.0180
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.7576 - loss: 0.0919
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.7576 - loss: 0.0101
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.7576 - loss: 0.0531
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.7576 - loss: 0.0180
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.7576 - loss: 0.0438
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7576 - loss: 0.0193
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.7576 - loss: 0.0402
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.7576 - loss: 0.0224
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.7576 - loss: 0.0309
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.7576 - loss: 0.0222
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.7576 - los